In [30]:
import numpy as np
class Board():
    def __init__(self, n=11):
        "Set up initial board configuration."
        self.n = n
        # Create the empty board array.
        self.empty_broad = np.zeros([self.n, self.n])
        for i in range(self.n // 2):
            self.empty_broad[i][11 - (5 - i):] = 10
            self.empty_broad[10 - i][:5 - i] = 10
        print(self.empty_broad)

    # add [][] indexer syntax to the Board
    def __getitem__(self, index):
        return self.empty_broad[index]

    def get_legal_moves(self, color):
        """Returns all the legal moves for the given color.
        (1 for white, -1 for black
        """
        moves = set()  # stores the legal moves.
        # Get all empty locations.
        for y in range(self.n):
            for x in range(self.n):
                if self[x][y] == 0:
                    moves.add((x, y))
        return list(moves)

    def has_legal_moves(self):
        """Returns True if has legal move else False
        """
        # Get all empty locations.
        for y in range(self.n):
            for x in range(self.n):
                if self[x][y] == 0:
                    return True
        return False

    def execute_move(self, move, color):
        """Perform the given move on the board; flips pieces as necessary.
        color gives the color pf the piece to play (1=white,-1=black)
        """
        (x,y) = move
        assert self[x][y] == 0
        self[x][y] = color

a=Board()
print( a.get_legal_moves(1))


[[ 0.  0.  0.  0.  0.  0. 10. 10. 10. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0. 10. 10. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 10. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10. 10.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10. 10. 10.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10. 10. 10. 10.  0.  0.  0.  0.  0.  0.]]
[(7, 3), (6, 9), (1, 6), (3, 7), (2, 5), (8, 5), (5, 8), (4, 0), (10, 8), (6, 7), (5, 5), (10, 7), (7, 6), (6, 10), (0, 4), (1, 1), (3, 2), (2, 6), (4, 5), (7, 5), (0, 1), (3, 1), (9, 9), (7, 8), (2, 1), (8, 9), (9, 4), (5, 1), (7, 2), (1, 5), (3, 6), (2, 2), (8, 6), (4, 1), (10, 9), (9, 7), (6, 4), (5, 4), (0, 5), (1, 0), (3, 5), (2, 7), (8, 3), (5, 10), (4, 6), (10, 10), (6, 1), (5, 7), (7, 4), (0, 2), (1, 3), (4, 8), (3, 0), (9, 8), (6, 2), (8, 10), (5, 0), (1, 4), (2, 3), (

In [16]:
from __future__ import print_function
import sys
sys.path.append('..')
from Game import Game
from GobangLogicnew import Board
import numpy as np

In [53]:
class GobangGame(Game):
    def __init__(self, n=11, nir=5):
        self.n = n
        self.n_in_row = nir

    def getInitBoard(self):
        # return initial board (numpy board)
        b = Board(self.n)
        return np.array(b.empty_broad)

    def getBoardSize(self):
        # (a,b) tuple
        return (self.n, self.n)

    def getActionSize(self):
        # return number of actions
        #return self.n + (self.n + self.n//2)/2 * 10
        return self.n * self.n

    def getNextState(self, board, player, action):
        # if player takes action on board, return next (board,player)
        # action must be a valid move
        if action == self.n * self.n:
            return (board, -player)
        b = Board(self.n)
        b.pieces = np.copy(board)
        move = (int(action / self.n), action % self.n)
        b.execute_move(move, player)
        return (b.pieces, -player)

    # modified
    def getValidMoves(self, board, player):
        # return a fixed size binary vector
        valids = [0] * self.getActionSize()
        b = Board(self.n)
        b.pieces = np.copy(board)
        legalMoves = b.get_legal_moves(player)
        if len(legalMoves) == 0:
            valids[-1] = 1
            return np.array(valids)
        for x, y in legalMoves:
            valids[self.n * x + y] = 1
        return np.array(valids)

    def check_single(self, line, player):
        L = len(line)
        opponent = player * (-1)
        for i in range(L-4):
            if player == line[i+1] == line[i+2] == line[i+3] == line[i+4]:
                if (i == 0 or line[i-1] != player) and (i == L-5 or line[i+5] != player):
                    return player
            if opponent == line[i+1] == line[i+2] == line[i+3] == line[i+4]:
                if (i == 0 or line[i-1] != opponent) and (i == L-5 or line[i+5] != opponent):
                    return opponent
        return 0

    # modified
    def getGameEnded(self, board, player):
        # return 0 if not ended, 1 if player 1 won, -1 if player 1 lost
        # player = 1
        b = Board(self.n)
        b.pieces = np.copy(board)
        # check if anybody win
        for i in range(6):
            winner = self.check_single(board[i, :i+6], player)
            if winner != 0:
                return winner
            winner = self.check_single(board[:i+6, i], player)
            if winner != 0:
                return winner
        for i in range(6, 11):
            winner = self.check_single(board[i, i-5:], player)
            if winner != 0:
                return winner
            winner = self.check_single(board[i-5:, i], player)
            if winner != 0:
                return winner
        for k in range(-5, 6):
            print(np.diag(board, k))
            winner = self.check_single(np.diag(board, k), player)
            if winner != 0:
                return winner
        #check if there is still legal moves
        if b.has_legal_moves():
            return 0
        # check if terminated when nobody win
        if not 0 in board:
            return 0
        return 0

    def getCanonicalForm(self, board, player):
        # return state if player==1, else return -state if player==-1
        new_board=player * board
        for i in range(self.n // 2):
            new_board[i][11 - (5 - i):] = 10
            new_board[10 - i][:5 - i] = 10
        return new_board

    # modified
    def getSymmetries(self, board, pi):
        # mirror, rotational
        assert(len(pi) == self.n**2 + 1)  # 1 for pass
        pi_board = np.reshape(pi[:-1], (self.n, self.n))
        l = []
        for i in range(1, 5):
            for j in [True, False]:
                newB = np.rot90(board, i)
                newPi = np.rot90(pi_board, i)
                if j:
                    newB = np.fliplr(newB)
                    newPi = np.fliplr(newPi)
                l += [(newB, list(newPi.ravel()) + [pi[-1]])]
        return l

    def stringRepresentation(self, board):
        return board.tostring()

    def display(self, board):
        n = board.shape[0]

        for y in range(n):
            print(y, "|", end="")
        print("")
        print(" -----------------------")
        for y in range(n):
            print(y, "|", end="")    # print the row #
            for x in range(n):
                piece = board[y][x]    # get the piece to print
                if piece == -1:
                    print("b ", end="")
                elif piece == 1:
                    print("W ", end="")
                else:
                    if x == n:
                        print("-", end="")
                    else:
                        print("- ", end="")
            print("|")
        print("   -----------------------")

121
[[ 0  1  0  0  0  0 10 10 10 10 10]
 [ 0  1  0  0  0  0  0 10 10 10 10]
 [ 0  1  0  0  0  0  0  0 10 10 10]
 [ 0  1  0  0  0  0  0  0  0 10 10]
 [ 0  1  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [10  0  0  0  0  0  0  0  0  0  0]
 [10 10  0  0  0  0  0  0  0  0  0]
 [10 10 10  0  0  0  0  0  0  0  0]
 [10 10 10 10  0  0  0  0  0  0  0]
 [10 10 10 10 10  0  0  0  0  0  0]]
[[ 0.  0.  0.  0.  0.  0. 10. 10. 10. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0. 10. 10. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 10. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 10. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10. 10.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10. 10. 10.  0.  0.  0.  0.  0.  0.  0.]
 [10. 10. 10. 10. 10.  0.  0.  0.  0.  0.  0.]]
1


In [45]:
a=GobangGame()
print(a.getActionSize())
board = np.zeros([11, 11]).astype(int)
for i in range(11//2):
    board[i][11-(5-i):] = 10
    board[10-i][:5-i] = 10

for i in range(0, 5):
    board[i][1] = 1
# board[4][1:6] = 1
print(board)
print(a.getGameEnded(board,-1))

In [ ]:
import numpy as np
class RandomPlayer():
    def __init__(self, game):
        self.game = game

    def play(self, board):
        a = np.random.randint(self.game.getActionSize())
        valids = self.game.getValidMoves(board, 1)
        while valids[a]!=1:
            a = np.random.randint(self.game.getActionSize())
        return a


class HumanGobangPlayer():
    def __init__(self, game):
        self.game = game

    def play(self, board):
        # display(board)
        valid = self.game.getValidMoves(board, 1)
        for i in range(len(valid)):
            if valid[i]:
                print(int(i/self.game.n), int(i%self.game.n))
        while True:
            a = input()

            x,y = [int(x) for x in a.split(' ')]
            a = self.game.n * x + y if x!= -1 else self.game.n ** 2
            if valid[a]:
                break
            else:
                print('Invalid')

        return a


class GreedyGobangPlayer():
    def __init__(self, game):
        self.game = game

    def play(self, board):
        valids = self.game.getValidMoves(board, 1)
        candidates = []
        for a in range(self.game.getActionSize()):
            if valids[a]==0:
                continue
            nextBoard, _ = self.game.getNextState(board, 1, a)
            score = self.game.getScore(nextBoard, 1)
            candidates += [(-score, a)]
        candidates.sort()
        return candidates[0][1]import numpy as np


class RandomPlayer():
    def __init__(self, game):
        self.game = game

    def play(self, board):
        a = np.random.randint(self.game.getActionSize())
        valids = self.game.getValidMoves(board, 1)
        while valids[a]!=1:
            a = np.random.randint(self.game.getActionSize())
        return a


class HumanGobangPlayer():
    def __init__(self, game):
        self.game = game

    def play(self, board):
        # display(board)
        valid = self.game.getValidMoves(board, 1)
        for i in range(len(valid)):
            if valid[i]:
                print(int(i/self.game.n), int(i%self.game.n))
        while True:
            a = input()

            x,y = [int(x) for x in a.split(' ')]
            a = self.game.n * x + y if x!= -1 else self.game.n ** 2
            if valid[a]:
                break
            else:
                print('Invalid')

        return a


class GreedyGobangPlayer():
    def __init__(self, game):
        self.game = game

    def play(self, board):
        valids = self.game.getValidMoves(board, 1)
        candidates = []
        for a in range(self.game.getActionSize()):
            if valids[a]==0:
                continue
            nextBoard, _ = self.game.getNextState(board, 1, a)
            score = self.game.getScore(nextBoard, 1)
            candidates += [(-score, a)]
        candidates.sort()
        return candidates[0][1]